In [5]:
from z3 import *

In [6]:
instance00 = {
    "m" : 1,
    "n" : 2,
    "l" : [15],
    "s" : [3, 2],
    "distances" : [
        [0, 4, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance0 = {
    "m" : 2,
    "n" : 2,
    "l" : [15, 10],
    "s" : [3, 2],
    "distances" : [
        [0, 3, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

instance1 = {
    "m" : 3,
    "n" : 7,
    "l" : [15, 10, 7],
    "s" : [3, 2, 6, 8, 5, 4, 4],
    "distances" : [
        [0, 3, 3, 6, 5, 6, 6, 2],
        [3, 0, 4, 3, 4, 7, 7, 3],
        [3, 4, 0, 7, 6, 3, 5, 3],
        [6, 3, 7, 0, 3, 6, 6, 4],
        [5, 4, 6, 3, 0, 3, 3, 3],
        [6, 7, 3, 6, 3, 0, 2, 4],
        [6, 7, 5, 6, 3, 2, 0, 4],
        [2, 3, 3, 4, 3, 4, 4, 0]
    ]
}

instance2 = {
    "m" : 2,
    "n" : 3,
    "l" : [15, 10],
    "s" : [10, 2, 4],
    "distances" : [
        [0, 3, 3, 1],
        [3, 0, 4, 1],
        [3, 4, 0, 1],
        [3, 2, 1, 0]
    ]
}

instance12 = {
    "m" : 5,
    "n" : 10,
    "l" : [10, 10, 19, 12, 11],
    "s" : [6, 3, 2, 6, 4, 5, 3, 6, 4, 3],
    "distances" : [
                [ 0, 5, 3, 2, 2, 2, 4, 2, 3, 1, 5 ], 
                [ 2, 0, 1, 1, 1, 2, 2, 2, 3, 4, 2 ], 
                [ 4, 2, 0, 4, 2, 2, 2, 5, 4, 1, 2 ], 
                [ 4, 2, 3, 0, 3, 3, 3, 1, 5, 4, 1 ], 
                [ 4, 2, 4, 4, 0, 1, 4, 5, 2, 5, 2 ], 
                [ 2, 4, 5, 1, 3, 0, 1, 3, 5, 4, 3 ], 
                [ 5, 4, 3, 3, 5, 4, 0, 3, 5, 1, 2 ], 
                [ 4, 3, 2, 2, 5, 5, 2, 0, 5, 5, 1 ], 
                [ 1, 5, 3, 4, 1, 1, 4, 1, 0, 2, 3 ], 
                [ 4, 2, 2, 2, 1, 3, 4, 3, 1, 0, 4 ], 
                [ 4, 5, 5, 4, 3, 5, 5, 1, 5, 5, 0 ] 
            ]
}

instance3 = {  
    "m" : 6,
    "n" : 12,
    "l" : [28,28,25,27,29,28],
    "s" : [7,13,5,8,6,10,4,9,3,5,3,13],
    "distances" : [
        [0,2,10,5,9,10,3,5,2,2,6,8,9],
        [2,0,6,7,6,10,4,9,8,8,9,5,1],
        [9,1,0,2,7,1,10,8,6,4,6,2,4],
        [10,4,4,0,3,9,8,2,2,6,9,8,2],
        [5,9,5,2,0,9,6,9,4,10,9,10,5],
        [8,2,10,7,6,0,10,4,5,3,4,3,1],
        [10,5,8,2,2,3,0,3,1,2,9,7,9],
        [5,9,4,4,10,7,10,0,5,8,8,6,2],
        [6,10,2,8,10,6,4,4,0,1,5,2,4],
        [6,3,6,7,1,2,3,4,1,0,10,9,10],
        [2,1,2,4,10,10,2,7,2,6,0,2,1],
        [10,1,4,3,2,8,4,1,1,9,7,0,10],
        [2,5,2,4,2,5,6,7,3,1,9,8,0]
    ]
}

In [17]:
def maxv(vs):
  m = vs[0]
  for v in vs[1:]:
    m = If(v > m, v, m)
  return m

def get_list_of_values(ll,j):
        return([If(x==j,1,0) for l in ll for x in l])

def mcp(instance):
    m = instance['m'] # couriers
    n = instance['n'] # packages
    l = instance['l'] # capacities
    s = instance['s'] # sizes
    
    o = Optimize()

    # define variables
    v = [[Int(f'v_{i}_{k}') for k in range(0,n+2)]for i in range(m)]
    d = [Int(f'd_{i}') for i in range(m)]

    distances = Array('distances', IntSort(), ArraySort(IntSort(), IntSort()))
    for j in range(n+1):
        for j1 in range(n+1):
            o.add(distances[j][j1] == instance['distances'][j][j1])

    s = Array('distances', IntSort(), IntSort())
    for j in range(n):
        o.add(s[j] == instance['s'][j]  )
    o.add(s[n] == 0)

    # define possible value for v[i][k]
    o.add([And(v[i][k] >= 0, v[i][k] <= n) for i in range(m) for k in range(1,n+1)])
    o.add([And(v[i][0] == n, v[i][n+1] == n) for i in range(m)])

    # for each i foreach k, each v[i][k] must be different, unless it is equal to n
    for j in range(n):
        o.add([Sum(get_list_of_values([[v[i][k] for k in range(1,n+1)] for i in range(m)],j))==1])
    
    # for each i, the sum of the weights of the packages carried by the courier i must be less than the capacity of the courier i
    for i in range(m):
        o.add(Sum([s[v[i][k]] for k in range(1,n+1)])<=l[i])

    # simmetry breaking
    # once a courier i return to the depot, it cant deliver other packages
    for i in range(m):
        for k in range(1,n+1):
            o.add(Implies(v[i][k]==n, v[i][k+1]==n))
    
    # lexycographic constraint between couriers with == capacity
    for i1 in range(m-1):
        for i2 in range(i1+1,m):
            o.add(Implies(l[i1]==l[i2], If(v[i1][1]!=n, v[i1][1], -1)<=If(v[i1][1]!=n, v[i1][1], -1)))
    
    # distances array
    for i in range(m):
        o.add(d[i] == Sum([distances[v[i][k]][v[i][k+1]] for k in range(0,n+1)]))

    o.minimize(maxv(d))
    res = o.check()
    #print(o.assertions())
    if res == sat:
        model = o.model()
        if False:
            for i in range(m):
                print()
                print(f"Courier {i+1}:")
                for k in range(n+2):
                    j = model.eval(v[i][k])
                    print(f"Time {k} Place {j}")
        else:
            for i in range(m):
                print()
                print(f"Courier {i+1}:")
                print(f"Distance: {model.eval(d[i])}")
        
        return o
    else:
        return unsat

In [18]:
instance = instance12
opt = mcp(instance)



Courier 1:
Distance: 8

Courier 2:
Distance: 8

Courier 3:
Distance: 8

Courier 4:
Distance: 5

Courier 5:
Distance: 8
